# Setup: BigQuery Data Ingestion (Safe Version)

This notebook safely sets up the BigQuery tables required for the Delivery Intelligence Agent workshop.

## Features
- ✅ Safe to run multiple times (idempotent)
- ✅ Checks if tables exist before creating
- ✅ Option to append or replace data
- ✅ Clear status messages throughout
- ✅ Automatic fallback to CSV loading if pyarrow is not available

## Requirements
- Google Cloud Project with BigQuery enabled
- Authentication set up (`gcloud auth application-default login`)
- Python packages: pandas, google-cloud-bigquery
- Optional but recommended: pyarrow (for faster uploads)

## Overview

We'll create 3 tables from the CSV data:
1. **delivery_orders** - Main table with core delivery information
2. **delivery_notes** - Customer and historical notes (separated due to long text)
3. **delivery_products** - Product SKUs (normalized from comma-delimited list)

## Configuration

Set your preferences here:

In [ ]:
# Configuration
PROJECT_ID = "traversaal-research"  # Change this to your project
DATASET_ID = "delivery_intelligence"
LOCATION = "US"  # Changed from us-central1 to US for BigQuery

# Data loading preferences
REPLACE_EXISTING_DATA = True  # Set to False to skip tables that already exist
SHOW_SAMPLE_DATA = True      # Set to False to skip sample queries at the end

## Clean Slate Setup

Before starting the workshop, let's clean up any existing output files from previous runs to ensure you see the actual pipeline execution:

In [ ]:
import os
import glob

# Configuration
CLEANUP_OUTPUT_FILES = True  # Set to False to skip cleanup

if CLEANUP_OUTPUT_FILES:
    print("🧹 Cleaning up existing output files from exercises...")
    print("=" * 60)
    
    # Define output files to clean up
    output_files = [
        # Exercise 1 outputs
        "../exercise_1_data_collection/collected_order_data.json",
        
        # Exercise 2 outputs
        "../exercise_2_risk_assessment/risk_assessment_output.json",
        "../risk_assessment_output.json",  # In case it was saved in parent directory
        "../exercise_2_risk_assessment/weather_mcp_server_demo.py",  # Demo file created by notebook
        
        # Exercise 3 outputs
        "../exercise_3_product_intelligence/product_intelligence_output.json",
        
        # Exercise 4 outputs
        "../exercise_4_communication_generation/communication_output.json",
        
        # Exercise 5 outputs
        "../exercise_5_final_integration/delivery_case_card.json",
        "../exercise_5_final_integration/delivery_case_card.md",
        "../exercise_5_final_integration/collected_order_data.json",
        "../exercise_5_final_integration/risk_assessment_output.json",
        "../exercise_5_final_integration/product_intelligence_output.json",
        "../exercise_5_final_integration/communication_output.json",
    ]
    
    # Also clean up any Jupyter checkpoint files
    checkpoint_patterns = [
        "../*/.ipynb_checkpoints/*",
        "../*/*/.ipynb_checkpoints/*"
    ]
    
    cleaned_count = 0
    
    # Clean specific output files
    for file_path in output_files:
        if os.path.exists(file_path):
            try:
                os.remove(file_path)
                print(f"✓ Removed: {os.path.basename(file_path)}")
                cleaned_count += 1
            except Exception as e:
                print(f"⚠️  Could not remove {file_path}: {e}")
        else:
            # File doesn't exist, which is fine
            pass
    
    # Clean checkpoint files
    for pattern in checkpoint_patterns:
        checkpoint_files = glob.glob(pattern)
        for checkpoint in checkpoint_files:
            try:
                os.remove(checkpoint)
                print(f"✓ Removed checkpoint: {os.path.basename(checkpoint)}")
                cleaned_count += 1
            except Exception as e:
                print(f"⚠️  Could not remove {checkpoint}: {e}")
    
    print("\n" + "=" * 60)
    if cleaned_count > 0:
        print(f"✅ Cleanup complete! Removed {cleaned_count} files.")
        print("\n💡 The exercises will now generate fresh outputs when you run them.")
    else:
        print("✅ No files to clean up - starting with a clean environment!")
        
    print("\n📝 Note: This cleanup ensures you see the actual pipeline execution")
    print("   rather than cached results from previous runs.")
else:
    print("ℹ️  Cleanup skipped (CLEANUP_OUTPUT_FILES=False)")
    print("   Existing output files will be preserved.")

## Environment Setup

In [ ]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    """Install a package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
        return True
    except subprocess.CalledProcessError:
        return False

# Check and install required packages
packages_to_install = {
    'pyarrow': 'pyarrow>=6.0.0',
    'google-cloud-bigquery': 'google-cloud-bigquery[pandas]',
    'pandas': 'pandas>=1.3.0',
    'db-dtypes': 'db-dtypes'  # Required for BigQuery datetime handling
}

print("Checking required packages...")
for package_name, package_spec in packages_to_install.items():
    try:
        __import__(package_name.replace('-', '_'))
        print(f"✅ {package_name} is already installed")
    except ImportError:
        print(f"📦 Installing {package_spec}...")
        if install_package(package_spec):
            print(f"✅ Successfully installed {package_name}")
        else:
            print(f"❌ Failed to install {package_name} - please install manually")
            print(f"   Run: pip install {package_spec}")

print("\n✅ Package check complete")

## Install Required Dependencies

Make sure you have the necessary packages installed:

In [ ]:
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.cloud.exceptions import Conflict, NotFound
import warnings
warnings.filterwarnings('ignore')

# Set up environment
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# Initialize BigQuery client
try:
    client = bigquery.Client(project=PROJECT_ID)
    print(f"✅ Connected to project: {PROJECT_ID}")
except Exception as e:
    print(f"❌ Failed to connect to BigQuery: {e}")
    print("\nPlease ensure:")
    print("1. You have the correct project ID")
    print("2. You are authenticated (run 'gcloud auth application-default login')")
    print("3. You have BigQuery API enabled")
    raise

## Helper Functions

In [ ]:
def table_exists(client, dataset_id, table_id):
    """Check if a table exists in BigQuery"""
    try:
        table_ref = f"{PROJECT_ID}.{dataset_id}.{table_id}"
        client.get_table(table_ref)
        return True
    except NotFound:
        return False
    except Exception as e:
        print(f"Error checking table {table_id}: {e}")
        return False

def safe_load_table(client, df, table_full_id, table_name):
    """Safely load data to BigQuery table"""
    # Check if table exists
    dataset_id = table_full_id.split('.')[1]
    table_id = table_full_id.split('.')[2]
    
    if table_exists(client, dataset_id, table_id):
        if REPLACE_EXISTING_DATA:
            print(f"⚠️  Table {table_name} exists - replacing data...")
            write_disposition = "WRITE_TRUNCATE"
        else:
            print(f"ℹ️  Table {table_name} exists - skipping (REPLACE_EXISTING_DATA=False)")
            return True
    else:
        print(f"📊 Creating new table {table_name}...")
        write_disposition = "WRITE_TRUNCATE"
    
    # Load data
    job_config = bigquery.LoadJobConfig(
        write_disposition=write_disposition,
        autodetect=True,
    )
    
    try:
        # Try pandas method first (requires pyarrow)
        try:
            job = client.load_table_from_dataframe(df, table_full_id, job_config=job_config)
            job.result()  # Wait for job to complete
            print(f"✅ Successfully loaded {len(df):,} rows into {table_name}")
            return True
        except Exception as pandas_error:
            if "pyarrow" in str(pandas_error):
                print("⚠️  Pyarrow not available, using CSV method instead...")
                
                # Alternative: Save to temporary CSV and load
                import tempfile
                import os
                
                with tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False) as tmp_file:
                    df.to_csv(tmp_file.name, index=False)
                    tmp_path = tmp_file.name
                
                # Load from CSV file
                with open(tmp_path, 'rb') as source_file:
                    job = client.load_table_from_file(
                        source_file, 
                        table_full_id, 
                        job_config=job_config
                    )
                    job.result()
                
                # Clean up temp file
                os.unlink(tmp_path)
                
                print(f"✅ Successfully loaded {len(df):,} rows into {table_name} (via CSV)")
                return True
            else:
                raise pandas_error
                
    except Exception as e:
        print(f"❌ Error loading {table_name}: {e}")
        return False

## Create or Verify Dataset

In [ ]:
# Create dataset if it doesn't exist
dataset_id = f"{PROJECT_ID}.{DATASET_ID}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = LOCATION
dataset.description = "Delivery Intelligence Workshop Data"

try:
    dataset = client.create_dataset(dataset, timeout=30)
    print(f"✅ Created new dataset: {dataset_id}")
except Conflict:
    print(f"ℹ️  Dataset {dataset_id} already exists - using existing dataset")
    dataset = client.get_dataset(dataset_id)
    
    # List existing tables
    tables = list(client.list_tables(dataset))
    if tables:
        print(f"\n📋 Existing tables in dataset:")
        for table in tables:
            table_ref = client.get_table(table.reference)
            print(f"   - {table.table_id} ({table_ref.num_rows:,} rows)")
    else:
        print("   (No existing tables)")
except Exception as e:
    print(f"❌ Error with dataset: {e}")
    raise

## Load CSV Data

In [ ]:
# Load the main data CSV
csv_path = '../db-data-csv/main-data.csv'

try:
    print(f"📁 Loading CSV from {csv_path}...")
    df = pd.read_csv(csv_path)
    print(f"✅ Loaded {len(df):,} rows")
    print(f"   Columns: {len(df.columns)}")
    print(f"   Memory usage: {df.memory_usage().sum() / 1024**2:.1f} MB")
except FileNotFoundError:
    print(f"❌ CSV file not found at {csv_path}")
    print("\nPlease ensure the CSV file exists at the correct path.")
    raise
except Exception as e:
    print(f"❌ Error loading CSV: {e}")
    raise

## Data Preparation

In [ ]:
print("🔧 Preparing data...")

# Fix any potential encoding issues in column names
df.columns = [col.replace('\ufeff', '') for col in df.columns]

# Convert date columns to proper datetime format
date_columns = ['SCHEDULED_DELIVERY_DATE', 'DELIVERY_CREATE_DATE']
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"   ✓ Converted {col} to datetime")

# Convert time columns to string format for BigQuery
time_columns = ['WINDOW_START', 'WINDOW_END']
for col in time_columns:
    if col in df.columns:
        df[col] = df[col].astype(str)
        print(f"   ✓ Converted {col} to string")

# Handle boolean columns
bool_columns = ['SPECIAL_ORDER', 'UNATTENDED_FLAG', 'PRO_XTRA_MEMBER', 
                'MANAGED_ACCOUNT', 'COMMERCIAL_ADDRESS_FLAG']
for col in bool_columns:
    if col in df.columns:
        df[col] = df[col].map({'TRUE': True, 'FALSE': False, True: True, False: False})
        print(f"   ✓ Converted {col} to boolean")

print("\n✅ Data preparation complete")

## Table 1: delivery_orders

In [ ]:
# Columns for the main delivery_orders table
delivery_orders_columns = [
    'DATA_ID', 'MARKET', 'SCHEDULED_DELIVERY_DATE', 'DELIVERY_CREATE_DATE',
    'VEHICLE_TYPE', 'CUSTOMER_ORDER_NUMBER', 'WORK_ORDER_NUMBER', 'SPECIAL_ORDER',
    'FLOC', 'FLOC_TYPE', 'SERVICE_TYPE', 'UNATTENDED_FLAG', 'WINDOW_START',
    'WINDOW_END', 'QUANTITY', 'VOLUME_CUBEFT', 'WEIGHT', 'PALLET',
    'FLOC_DELIVERY_ATTEMPTS_LAST_15_DAYS', 'FLOC_OTC_FAILURES_LAST_15_DAYS',
    'FLOC_OTC_FAILURE_PCT_LAST_15_DAYS', 'DLVRY_RISK_DECILE', 'DLVRY_RISK_BUCKET',
    'DLVRY_RISK_PERCENTILE', 'DLVRY_RISK_TOP_FEATURE', 'DESTINATION_ADDRESS',
    'STREET_VIEW_URL', 'COMMERCIAL_ADDRESS_FLAG', 'BUSINESS_HOURS',
    'STRT_VW_IMG_DSCRPTN', 'WTHR_CATEGORY', 'PRECIPITATION', 'CUSTOMER_NAME',
    'PRO_XTRA_MEMBER', 'MANAGED_ACCOUNT', 'OSR_NAME', 'RI_GENERATE_DATETIME'
]

# Create delivery_orders dataframe
print("\n📊 Processing delivery_orders table...")
df_delivery_orders = df[delivery_orders_columns].copy()
table_id = f"{PROJECT_ID}.{DATASET_ID}.delivery_orders"

# Load to BigQuery
safe_load_table(client, df_delivery_orders, table_id, "delivery_orders")

## Table 2: delivery_notes

In [ ]:
# Columns for the delivery_notes table
delivery_notes_columns = [
    'DATA_ID', 'CUSTOMER_NOTES', 'CUSTOMER_NOTES_LLM_SUMMARY',
    'HISTORIC_NOTES_LLM_SUMMARY', 'CUSTOMER_NOTES_KEY_WORDS',
    'HISTORIC_NOTES_W_LABELS'
]

# Create delivery_notes dataframe
print("\n📊 Processing delivery_notes table...")
df_delivery_notes = df[delivery_notes_columns].copy()
table_id = f"{PROJECT_ID}.{DATASET_ID}.delivery_notes"

# Load to BigQuery
safe_load_table(client, df_delivery_notes, table_id, "delivery_notes")

## Table 3: delivery_products

In [ ]:
# Process SKU data - split comma-delimited lists into separate rows
print("\n📊 Processing delivery_products table...")
print("   Splitting comma-delimited SKUs into individual rows...")

product_rows = []
skipped_count = 0

for idx, row in df.iterrows():
    data_id = row['DATA_ID']
    sku_desc = row['SKU_DESCRIPTION']
    
    if pd.notna(sku_desc) and sku_desc:
        # Split by ' ,|' which seems to be the delimiter
        skus = [sku.strip() for sku in str(sku_desc).split(' ,|')]
        
        for sku in skus:
            if sku:  # Only add non-empty SKUs
                product_rows.append({
                    'DATA_ID': data_id,
                    'SKU_DESCRIPTION': sku
                })
    else:
        skipped_count += 1

# Create products dataframe
df_delivery_products = pd.DataFrame(product_rows)
print(f"   Created {len(df_delivery_products):,} product rows from {len(df):,} deliveries")
if skipped_count > 0:
    print(f"   Skipped {skipped_count} deliveries with no products")

# Load to BigQuery
table_id = f"{PROJECT_ID}.{DATASET_ID}.delivery_products"
safe_load_table(client, df_delivery_products, table_id, "delivery_products")

## Verify Setup

In [ ]:
# List all tables in the dataset with their stats
print("\n📊 Final table summary:")
print("=" * 60)

tables = list(client.list_tables(dataset))
total_size = 0
total_rows = 0

for table in tables:
    try:
        table_ref = client.get_table(table.reference)
        size_mb = table_ref.num_bytes / 1024 / 1024
        total_size += size_mb
        total_rows += table_ref.num_rows
        
        print(f"\n📌 {table.table_id}")
        print(f"   Rows: {table_ref.num_rows:,}")
        print(f"   Size: {size_mb:.2f} MB")
        print(f"   Columns: {len(table_ref.schema)}")
        print(f"   Created: {table_ref.created}")
    except Exception as e:
        print(f"\n❌ Error getting info for {table.table_id}: {e}")

print("\n" + "=" * 60)
print(f"Total: {len(tables)} tables, {total_rows:,} rows, {total_size:.2f} MB")

## Sample Queries (Optional)

In [ ]:
if SHOW_SAMPLE_DATA:
    print("\n🔍 Running sample queries...\n")
    
    # Query 1: Sample from delivery_orders
    try:
        query1 = f"""
        SELECT 
            DATA_ID,
            CUSTOMER_ORDER_NUMBER,
            SCHEDULED_DELIVERY_DATE,
            VEHICLE_TYPE,
            DLVRY_RISK_BUCKET
        FROM `{PROJECT_ID}.{DATASET_ID}.delivery_orders`
        LIMIT 3
        """
        df_sample = client.query(query1).to_dataframe()
        print("📋 Sample from delivery_orders:")
        print(df_sample)
    except Exception as e:
        print(f"❌ Error querying delivery_orders: {e}")
    
    # Query 2: Check for notes
    try:
        query2 = f"""
        SELECT 
            COUNT(*) as total_deliveries,
            COUNTIF(CUSTOMER_NOTES IS NOT NULL) as deliveries_with_notes
        FROM `{PROJECT_ID}.{DATASET_ID}.delivery_notes`
        """
        df_notes_stats = client.query(query2).to_dataframe()
        print("\n📋 Notes statistics:")
        print(df_notes_stats)
    except Exception as e:
        print(f"❌ Error querying delivery_notes: {e}")
    
    # Query 3: Product counts
    try:
        query3 = f"""
        SELECT 
            COUNT(DISTINCT DATA_ID) as unique_deliveries,
            COUNT(*) as total_products,
            ROUND(COUNT(*) / COUNT(DISTINCT DATA_ID), 2) as avg_products_per_delivery
        FROM `{PROJECT_ID}.{DATASET_ID}.delivery_products`
        """
        df_product_stats = client.query(query3).to_dataframe()
        print("\n📋 Product statistics:")
        print(df_product_stats)
    except Exception as e:
        print(f"❌ Error querying delivery_products: {e}")
else:
    print("\nℹ️  Sample queries skipped (SHOW_SAMPLE_DATA=False)")

## Summary

### ✅ Setup Complete!

The following tables are now available in BigQuery:
1. **delivery_orders** - Main delivery data
2. **delivery_notes** - Customer and historical notes
3. **delivery_products** - Individual product SKUs

### Next Steps

1. **Update Exercise notebooks** to use these tables:
   - `{PROJECT_ID}.{DATASET_ID}.delivery_orders`
   - `{PROJECT_ID}.{DATASET_ID}.delivery_notes`
   - `{PROJECT_ID}.{DATASET_ID}.delivery_products`

2. **Common queries for agents**:
   ```sql
   -- Get complete order info
   SELECT o.*, n.CUSTOMER_NOTES
   FROM `delivery_orders` o
   LEFT JOIN `delivery_notes` n USING(DATA_ID)
   WHERE o.DATA_ID = ?
   
   -- Get products for an order
   SELECT SKU_DESCRIPTION 
   FROM `delivery_products`
   WHERE DATA_ID = ?
   ```

3. **Test with workshop notebooks** to ensure agents can query effectively

### Troubleshooting

If you encounter issues:
- Check the PROJECT_ID is correct
- Ensure you have BigQuery permissions
- Verify the CSV file path is correct
- Check REPLACE_EXISTING_DATA setting if tables already exist